# 使用训练工具
## 训练工具介绍
huggingface 提供了训练工具，同意了模型再训练的过程，使调用者无需了解模型的具体计算过程，只需要针对具体的任务准备好数据集，可以再训练模型。
## 模型训练
### 1. 准备数据


In [1]:
!pip install transformers
!pip install transformers[torch]
!pip install datasets
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# !nvcc -V

Looking in indexes: https://download.pytorch.org/whl/cu118


In [2]:
import torch
print(torch.cuda.is_available())
print(torch.__version__)

True
2.0.1+cu118


In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("hfl/rbt3")
# 文本编码
tokenizer.batch_encode_plus(
    ['明月装饰了我的窗子','你装饰了别人的梦'],
    truncation = True,
)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': [[101, 3209, 3299, 6163, 7652, 749, 2769, 4638, 4970, 2094, 102], [101, 872, 6163, 7652, 749, 1166, 782, 4638, 3457, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

### 2. 准备数据集
ChnSentiCorp 情感分类文本数据

In [66]:

from datasets import load_dataset
dataset = load_dataset("lansinuote/ChnSentiCorp")
# 缩小数据规模，便于测试
dataset['train'] = dataset['train'].shuffle().select(range(4000))
dataset['test'] = dataset['test'].shuffle().select(range(400))
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 4000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1200
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 400
    })
})

数据集转换将文本数据转换成数字

In [67]:
def f(data):
  return tokenizer.batch_encode_plus(data['text'], truncation=True)

dataset = dataset.map(
    f,
    batched = True,
    batch_size = 1000,
    num_proc = 4,
    remove_columns = ['text']
)
dataset

Map (num_proc=4):   0%|          | 0/4000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/400 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4000
    })
    validation: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1200
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 400
    })
})

hfl/rbt3 模型不能处理长度超过 512 个词的句子，过滤超出上限的句子

In [68]:
def f(data):
  return [len(i) < 512 for i in data['input_ids']]

dataset=dataset.filter(f, batched =True, batch_size=1000, num_proc=4)
dataset

Filter (num_proc=4):   0%|          | 0/4000 [00:00<?, ? examples/s]

Filter (num_proc=4):   0%|          | 0/400 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3949
    })
    validation: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1190
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 399
    })
})

In [15]:
# 查看一个句子编码后的内容
for k,v in dataset['train'][1].items():
  print(k, ':', v)

print(len(dataset['train'][1]['input_ids']))

label : 0
input_ids : [101, 2769, 3777, 1298, 722, 3180, 857, 6814, 3297, 2345, 4638, 6983, 2421, 8024, 6983, 2421, 2399, 807, 719, 6823, 8024, 7478, 2382, 3655, 3191, 8024, 2791, 7313, 1045, 5296, 738, 679, 1962, 8024, 3819, 2797, 7313, 3300, 2460, 1456, 8024, 7370, 749, 4895, 4685, 1744, 2191, 6818, 809, 1912, 8024, 4895, 1071, 800, 3250, 4157, 6963, 679, 3221, 2523, 6818, 511, 1377, 5543, 2458, 2196, 6821, 702, 1814, 2356, 3315, 6716, 1168, 1905, 6963, 3221, 4129, 2212, 8024, 6983, 2421, 738, 1962, 1008, 1168, 1905, 6963, 4129, 5885, 5885, 511, 1048, 6589, 5632, 1221, 3193, 7623, 3766, 3300, 784, 720, 1391, 4638, 8024, 679, 6814, 6820, 5050, 1391, 2533, 7653, 102]
token_type_ids : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

### 3. 定义模型和训练工具

In [7]:
from transformers import AutoModelForSequenceClassification
# !pip install torch
import torch
model = AutoModelForSequenceClassification.from_pretrained('hfl/rbt3', num_labels=2)
# 查看模型参数量
sum([i.nelement() for i in model.parameters()])/10000

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/rbt3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


3847.8338

对模型进行一次试算，观察模型的输出

In [16]:
# 模拟一批数据
test_data = {
    "input_ids": torch.ones(4,3, dtype=torch.long),
    "token_type_ids": torch.ones(4,3, dtype=torch.long),
    "attention_mask": torch.ones(4,3, dtype=torch.long),
    "labels": torch.ones(4, dtype=torch.long)
}
print(test_data)
out = model(**test_data)
print(out)

{'input_ids': tensor([[1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]]), 'token_type_ids': tensor([[1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]]), 'labels': tensor([1, 1, 1, 1])}
SequenceClassifierOutput(loss=tensor(0.5362, grad_fn=<NllLossBackward0>), logits=tensor([[-0.4414, -0.0981],
        [-0.4414, -0.0981],
        [-0.4414, -0.0981],
        [-0.4414, -0.0981]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


定义评价指标，情感分类问题往往关注准确率，加载准确率指标。

In [10]:
from datasets import load_metric
metric = load_metric('accuracy')


<ipython-input-10-bbd4ecc85a80>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('accuracy')


由于模型计算的输出和评价指标要求的输入还有差别，所以需要定义一个转换函数进行计算

In [11]:
import numpy as np
from transformers.trainer_utils import EvalPrediction
def compute_metrics(eval_pred):
  logits, labels = eval_pred
  logits = logits.argmax(axis=1)
  return metric.compute(predictions=logits, references= labels)

# 模拟输出
eval_pred = EvalPrediction(
    predictions= np.array([[0,1],[1,2],[2,3],[3,4]]),
    label_ids = np.array([1,1,0,1]),
)

compute_metrics(eval_pred)

{'accuracy': 0.75}

定义训练参数

In [69]:
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

# !pip install accelerate==0.20.3
# !pip install accelerate -U
# !pip install accelerate==0.20.1
from transformers import TrainingArguments
args = TrainingArguments(
    output_dir = "/content/output_dir",
    evaluation_strategy='steps',
    eval_steps = 30,
    save_strategy = 'steps',
    save_steps = 30,
    num_train_epochs = 3,
    learning_rate = 1e-4,
    weight_decay=1e-2,
    per_device_eval_batch_size=16,
    per_device_train_batch_size=16,
    no_cuda = False,
)

定义训练器

In [70]:
from transformers import Trainer
from transformers.data.data_collator import DataCollatorWithPadding
trainer = Trainer(
    model = model,
    args = args,
    train_dataset = dataset['train'],
    eval_dataset = dataset['validation'],
    compute_metrics = compute_metrics,
    data_collator = DataCollatorWithPadding(tokenizer),
)

数据整理函数

In [78]:
data_collator = DataCollatorWithPadding(tokenizer)
data = dataset['train'][:5]
for i in data["input_ids"]:
  print(len(i))
# 调用数据集整理函数
data = data_collator(data)
for k,v in data.items():
  print(k,v.shape)

144
166
77
43
75
input_ids torch.Size([5, 166])
token_type_ids torch.Size([5, 166])
attention_mask torch.Size([5, 166])
labels torch.Size([5])


In [22]:
tokenizer.decode(data['input_ids'][0])

'[CLS] 降 价 太 快 ~ 早 找 到 就 多 等 几 天 了 才 一 个 月 就 降 了 500 现 在 还 送 无 线 路 由 器 。 哭 ~ ~ ~ [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

### 训练模型
在模型开始训练前评估模型，定下训练基准，训练结束后再比对基准，已验证训练的有效性

In [71]:
trainer.evaluate()

{'eval_loss': 0.4071004390716553,
 'eval_accuracy': 0.8344537815126051,
 'eval_runtime': 4.85,
 'eval_samples_per_second': 245.359,
 'eval_steps_per_second': 15.464}

模型训练

In [72]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
30,No log,0.395486,0.847899
60,No log,0.367246,0.851261
90,No log,0.407896,0.843697
120,No log,0.401315,0.836134
150,No log,0.425302,0.835294
180,No log,0.392401,0.852941
210,No log,0.432099,0.848739
240,No log,0.384216,0.837815
270,No log,0.366222,0.863866
300,No log,0.353009,0.875630


TrainOutput(global_step=741, training_loss=0.3249004173536211, metrics={'train_runtime': 297.7382, 'train_samples_per_second': 39.79, 'train_steps_per_second': 2.489, 'total_flos': 412217803310076.0, 'train_loss': 0.3249004173536211, 'epoch': 3.0})

评估模型

In [73]:
trainer.evaluate()

{'eval_loss': 0.3668989837169647,
 'eval_accuracy': 0.8697478991596639,
 'eval_runtime': 5.3264,
 'eval_samples_per_second': 223.416,
 'eval_steps_per_second': 14.081,
 'epoch': 3.0}

模型保存

In [74]:
trainer.save_model(output_dir="/content/save_model")

In [76]:
!ls -l /content/save_model

total 150336
-rw-r--r-- 1 root root       896 Aug  3 10:45 config.json
-rw-r--r-- 1 root root 153932380 Aug  3 10:45 pytorch_model.bin
-rw-r--r-- 1 root root      3963 Aug  3 10:45 training_args.bin


加载保存的模型

In [77]:
import torch
model.load_state_dict(torch.load("/content/save_model/pytorch_model.bin"))

<All keys matched successfully>

使用保存的模型进行预测

In [80]:
model.eval()
for i,data in enumerate(trainer.get_eval_dataloader()):
  break

out = model(**data)
out = out["logits"].argmax(dim=1)
for i in range(8):
  print(tokenizer.decode(data["input_ids"][i],skip_special_tokens = True))
  print("label=",data['labels'][i].item())
  print("predict=",out[i].item())

這 間 酒 店 環 境 和 服 務 態 度 亦 算 不 錯, 但 房 間 空 間 太 小 ~ ~ 不 宣 容 納 太 大 件 行 李 ~ ~ 且 房 間 格 調 還 可 以 ~ ~ 中 餐 廳 的 廣 東 點 心 不 太 好 吃 ~ ~ 要 改 善 之 ~ ~ ~ ~ 但 算 價 錢 平 宜 ~ ~ 可 接 受 ~ ~ 西 餐 廳 格 調 都 很 好 ~ ~ 但 吃 的 味 道 一 般 且 令 人 等 得 太 耐 了 ~ ~ 要 改 善 之 ~ ~
label= 1
predict= 0
< 荐 书 > 推 荐 所 有 喜 欢 < 红 楼 > 的 红 迷 们 一 定 要 收 藏 这 本 书, 要 知 道 当 年 我 听 说 这 本 书 的 时 候 花 很 长 时 间 去 图 书 馆 找 和 借 都 没 能 如 愿, 所 以 这 次 一 看 到 当 当 有, 马 上 买 了, 红 迷 们 也 要 记 得 备 货 哦!
label= 1
predict= 0
商 品 的 不 足 暂 时 还 没 发 现 ， 京 东 的 订 单 处 理 速 度 实 在....... 周 二 就 打 包 完 成 ， 周 五 才 发 货...
label= 0
predict= 0
２００１ 年 来 福 州 就 住 在 这 里 ， 这 次 感 觉 房 间 就 了 点 ， 温 泉 水 还 是 有 的 ． 总 的 来 说 很 满 意 ． 早 餐 简 单 了 些 ．
label= 1
predict= 1
不 错 的 上 网 本 ， 外 形 很 漂 亮 ， 操 作 系 统 应 该 是 个 很 大 的 卖 点 ， 电 池 还 可 以 。 整 体 上 讲 ， 作 为 一 个 上 网 本 的 定 位 ， 还 是 不 错 的 。
label= 1
predict= 1
房 间 地 毯 太 脏 ， 临 近 火 车 站 十 分 吵 闹 ， 还 好 是 双 层 玻 璃 。 服 务 一 般 ， 酒 店 门 口 的 taxi 讲 是 酒 店 的 长 期 合 作 关 系 ， 每 月 要 交 费 给 酒 店 。 从 酒 店 到 机 场 讲 得 是 打 表 147 元 ， 到 了 后 非 要 200 元 ， 可 能 被 小 宰 30 - 40 元 。
label= 0
predict= 0
本 来 想 没 事 的 时 候 